In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import scipy as sc
import matplotlib.pyplot as plt

import itertools
from sklearn.metrics import confusion_matrix

import operator

%matplotlib inline
import numpy as np
import pandas as np
import scipy
import matplotlib.pyplot as plt
from scipy import stats
import math

from python_utils import *
import pandas as pd
import numpy as np
import math
import random
#from __future__ import division

# Functions

In [2]:
def bootstrap_ci(df_aux, variable, div, classes, repetitions = 3000, alpha = 0.05, random_state=np.arange(1,30000,10)):
    # df: a data frame that includes observations of the two sample
    # variable: the column name of the column that includes observations
    # classes: the column name of the column that includes group assignment (This column should contain two different group names)
    # repetitions: number of times you want the bootstrapping to repeat. Default is 1000.
    # alpha: likelihood that the true population parameter lies outside the confidence interval. Default is 0.05. 
    # random_stata: enable users to set their own random_state, default is None. 
    
    
    # keep the variable that we want to study and the label to be able to divide the population.
    df = df_aux[[variable, classes]]
    bootstrap_sample_size = len(df) 
    
    mean_diffs = []
    print("sample size: ", int(bootstrap_sample_size/div))
    for i in range(repetitions):
        # Sample patients randomly
        bootstrap_sample = df.sample(n = int(bootstrap_sample_size/div), replace = True, random_state = random_state[i])
        # Mean
        mean_sample = bootstrap_sample.groupby(classes).sum() / bootstrap_sample.groupby(classes).count()
        mean_diff = mean_sample.iloc[1,0] - mean_sample.iloc[0,0]
        mean_diffs.append(mean_diff)

    # confidence interval
    left = np.percentile(mean_diffs, alpha/2*100)
    right = np.percentile(mean_diffs, 100-alpha/2*100)    
    out = [round(left,2), round(right,2)]
    
    return out

# Load data

In [3]:
i = 0
n = 4

X_train = np.load("../../../ORIGINAL_DATA/MDR/splits_14_days/notbalanced/split_" + str(i) + "/X_train_tensor_" + str(n)+ ".npy")

y_train = pd.read_csv("../../../ORIGINAL_DATA/MDR/splits_14_days/notbalanced/split_" + str(i) + "/y_train_" + str(n)+ ".csv", index_col=0)

X_val = np.load("../../../ORIGINAL_DATA/MDR/splits_14_days/notbalanced/split_" + str(i) + "/X_val_tensor_" + str(n)+ ".npy")

y_val = pd.read_csv("../../../ORIGINAL_DATA/MDR/splits_14_days/notbalanced/split_" + str(i) + "/y_val_" + str(n)+ ".csv", index_col=0)


X = np.append(X_train, X_val, axis=0)
X = np.where(X !=666, X, 0)
X = X.reshape(X.shape[0], -1)

y = y_train.append(y_val)

/tmp/ipykernel_1055658/270437044.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  y = y_train.append(y_val)


In [4]:
temporaryKeys = ['AMG', 'ATF', 'CAR', 'CF1', 'CF2', 'CF3',
       'CF4', 'Falta', 'GCC', 'GLI', 'LIN', 'LIP', 'MAC', 'MON', 'NTI', 'OTR',
       'OXA', 'PAP', 'PEN', 'POL', 'QUI', 'SUL', 'TTC', 'pc_acinet',
       'pc_enterob', 'pc_enteroc', 'pc_pseud', 'pc_staph', 'pc_stenot',
       'pc_no_germ', 'isVM', 'numberOfPatients', 'numberOfPatientsMR',
       'neighbor_AMG', 'neighbor_ATF', 'neighbor_CAR', 'neighbor_CF1',
       'neighbor_CF2', 'neighbor_CF3', 'neighbor_CF4', 'neighbor_Falta',
       'neighbor_GCC', 'neighbor_GLI', 'neighbor_LIN', 'neighbor_LIP',
       'neighbor_MAC', 'neighbor_MON', 'neighbor_NTI', 'neighbor_OTR',
       'neighbor_OXA', 'neighbor_PAP', 'neighbor_PEN', 'neighbor_POL',
       'neighbor_QUI', 'neighbor_SUL', 'neighbor_TTC']

keys_2 = []
for i in range(len(temporaryKeys)):
    keys_2.extend([temporaryKeys[i] + str(s) for s in np.arange(0, 14, 1).tolist()])

X = pd.DataFrame(data=X, columns=keys_2)

In [5]:
len(temporaryKeys)

56

In [6]:
X.keys().values

array(['AMG0', 'AMG1', 'AMG2', 'AMG3', 'AMG4', 'AMG5', 'AMG6', 'AMG7',
       'AMG8', 'AMG9', 'AMG10', 'AMG11', 'AMG12', 'AMG13', 'ATF0', 'ATF1',
       'ATF2', 'ATF3', 'ATF4', 'ATF5', 'ATF6', 'ATF7', 'ATF8', 'ATF9',
       'ATF10', 'ATF11', 'ATF12', 'ATF13', 'CAR0', 'CAR1', 'CAR2', 'CAR3',
       'CAR4', 'CAR5', 'CAR6', 'CAR7', 'CAR8', 'CAR9', 'CAR10', 'CAR11',
       'CAR12', 'CAR13', 'CF10', 'CF11', 'CF12', 'CF13', 'CF14', 'CF15',
       'CF16', 'CF17', 'CF18', 'CF19', 'CF110', 'CF111', 'CF112', 'CF113',
       'CF20', 'CF21', 'CF22', 'CF23', 'CF24', 'CF25', 'CF26', 'CF27',
       'CF28', 'CF29', 'CF210', 'CF211', 'CF212', 'CF213', 'CF30', 'CF31',
       'CF32', 'CF33', 'CF34', 'CF35', 'CF36', 'CF37', 'CF38', 'CF39',
       'CF310', 'CF311', 'CF312', 'CF313', 'CF40', 'CF41', 'CF42', 'CF43',
       'CF44', 'CF45', 'CF46', 'CF47', 'CF48', 'CF49', 'CF410', 'CF411',
       'CF412', 'CF413', 'Falta0', 'Falta1', 'Falta2', 'Falta3', 'Falta4',
       'Falta5', 'Falta6', 'Falta7', 'Falta8', 

# Code

In [7]:
params = X.keys().values
X["individualMRGerm"] = y.individualMRGerm.values # Label values


percentage_resampling = 0.8
importances_variables = np.zeros((len(params), 2))
for i in range(len(params)):
    importances_variables[i,:] = bootstrap_ci(X, params[i], (1 / percentage_resampling),
                                              'individualMRGerm', repetitions=3000)

sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:  2020
sample size:

In [8]:
df_aux = pd.DataFrame(importances_variables)
df_aux["selected"] = "Y"
logicExpression = (df_aux[0] == 0) | (df_aux[1] == 0)   
df_aux["selected"] = df_aux["selected"].where(~logicExpression, "N")
logicExpression = np.sign(df_aux[0]) != np.sign(df_aux[1])
df_aux["selected"] = df_aux["selected"].where(~logicExpression, "N")
df_aux["features"] = params

In [ ]:
df_aux.to_csv("results_bootstrapping.csv")